<a href="https://colab.research.google.com/github/emilstahl97/Scalable-Machine-Learning-and-Deep-Learning-ID2223/blob/notebooks/lab2_id2223.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Add dependencies

In [ ]:
!pip install sentence_transformers
!pip install transformers
!pip install tokenizers
!pip install torch
!pip install wget
!pip install pyspark

     |████████████████████████████████| 78 kB 3.1 MB/s 
     |████████████████████████████████| 3.3 MB 12.1 MB/s 
     |████████████████████████████████| 3.3 MB 32.8 MB/s 
     |████████████████████████████████| 1.2 MB 36.6 MB/s 
     |████████████████████████████████| 61 kB 411 kB/s 
     |████████████████████████████████| 895 kB 40.3 MB/s 
     |████████████████████████████████| 596 kB 44.5 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=0bab80c95dca4d140669fa3d974b07800b312f170be6f4a67a5f8721801ac806
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84da70092ea526466eadc2bfb197c4bcb8d4fa5f7bad
Successfully built sentence-transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9672 sha256=75954d40425ee430efca11b9500c395652e9d3633a

Import libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import StringIndexer


import os
import re
import csv
import wget
import json
import math
import scipy
import torch
import string
import sklearn

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Input

from sentence_transformers import SentenceTransformer
from sentence_transformers import LoggingHandler
from sentence_transformers import models, losses, util
from sentence_transformers.readers import InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

from transformers import BertTokenizer, TFBertModel, BertConfig
from tokenizers import BertWordPieceTokenizer

from torch.utils.data import DataLoader

from datetime import datetime

**Mount Google Drive to load saved models**

In [ ]:
# README - Execute this cell to mount the notebook in your google drive. 
# Execute the cell and follow the link to sign and, paste the given key in the little text box. The credentials are only available for you. 

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

if not os.path.exists("/content/drive/MyDrive/stsbenchmark"): 
  os.mkdir("/content/drive/MyDrive/stsbenchmark")
  os.chdir("/content/drive/MyDrive/stsbenchmark")
  !git clone https://github.com/emilstahl97/stsbenchmark.git
  !git pull
else:
  print("Dataset exists")
  os.chdir("/content/drive/MyDrive/stsbenchmark")

train_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-train.csv"
test_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-test.csv"
dev_path = "/content/drive/MyDrive/stsbenchmark/stsbenchmark/sts-dev.csv"


Mounted at /content/drive
Dataset exists


## **REGRESSION**

In [ ]:
model_name = 'bert-base-uncased'
word_embedding_model = models.Transformer(model_name)

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
columns = ['title', 'type', 'year', 'id', 'score', 'sentence_1', 'sentence_2']

In [ ]:

train_samples = []
test_samples = []
dev_samples = []

paths = {"train": train_path, "test": test_path, "dev": dev_path}

for key, path in paths.items():
  with open(path, newline='') as f:
    temp = csv.DictReader(f, delimiter='\t', fieldnames=columns, quoting=csv.QUOTE_NONE)
    for row in temp:
        score = float(row['score']) / 2.5 - 1 
        input_example = InputExample(texts=[row['sentence_1'], row['sentence_2']], label=score)
        if (key == "train"):
          train_samples.append(input_example)
        elif key == "test":
          test_samples.append(input_example)
        elif key == "dev":
          dev_samples.append(input_example)
        else:
          raise Exception(f"key {key} not know, exiting")



Considering the given paper "*Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks*"

In [ ]:
train_batch_size = 16
learn_rate = 2e-5
num_epochs = 1

Mean-pooling strategy

In [ ]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

Define the model

In [ ]:
# custom model using mean pooling of the word embeddings given as input
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Load the training set and define the loss function as the cosine similarity

In [ ]:
train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

Define the evaluator for the sentence embeddings

In [ ]:
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, batch_size=train_batch_size, name='sts-dev')

10% of train dataset for warm-up

In [ ]:
warmup_steps = math.ceil(len(train_dataloader) * num_epochs * 0.1)

**Training**

In [ ]:
save_path = './training_sts_reg_'+model_name+'-'+datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

In [ ]:
model.fit(train_objectives=[(train_dataloader, train_loss)],
            optimizer_class=torch.optim.Adam,
            optimizer_params={'lr': learn_rate},
            evaluator=evaluator,
            epochs=num_epochs,
            evaluation_steps=1000,
            warmup_steps=warmup_steps,
            output_path=save_path)

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/360 [00:00<?, ?it/s]

**Evaluation on STS benchmark dataset**

Mathematical relationship: *cosine_similarity = 1 - cosine_distance*

In [ ]:
print('Loading the stored model ...')
model = SentenceTransformer(save_path)

In [ ]:
test_eval = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, batch_size=train_batch_size, name='sts-test')
c_s = test_eval(model, output_path=save_path)
print('Cosine similarity with the sentence_transformers library = ', c_s)

# sometimes the result is between 0.7 and 0.8

Embedding sentences

In [ ]:
df_test = pd.read_csv(test_path, sep='\t', header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df_test.columns = columns

In [ ]:
embed_1 = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2 = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

Compute the cosine similarity

In [ ]:
cos_sim = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1, embed_2)
print('Cosine similarity = ', cos_sim)

Spearmean correlation coefficient

In [ ]:
spr_corr = scipy.stats.spearmanr(cos_sim, df_test['score'])
print('Spearmean correlation coefficient = ', spr_corr[0])

**Comment:** the two results match each other

## *S-BERT classification objective*

In [ ]:
print('***** Downloading dataset ...')
wget.download('https://nlp.stanford.edu/projects/snli/snli_1.0.zip', './snli_1.0.zip')
!unzip snli_1.0.zip

spark = SparkSession.builder.getOrCreate()
train_class_path = 'snli_1.0/snli_1.0_train.jsonl'
train_class = spark.read.json(train_class_path)

test_class_path = 'snli_1.0/snli_1.0_test.jsonl'
test_class = spark.read.json(test_class_path)

dev_class_path = 'snli_1.0/snli_1.0_dev.jsonl'
dev_class = spark.read.json(dev_class_path)

In [ ]:

indexer = StringIndexer(inputCol="gold_label", outputCol="label")

In [ ]:
def CreatInputExampleList(df):
    samples = []
    for index, row in df.iterrows():
        input_example = InputExample(texts=[row['sentence1'], row['sentence2']], label=row['label'])
        samples.append(input_example)
    return samples
    
def CreatClassSamples(df):
    df = df.filter(col("gold_label") != "-")
    df = indexer.fit(df).transform(df)
    df = df.withColumn("label", col("label").cast('int'))

    df_class = df.select("sentence1", "sentence2", "label").toPandas()

    samples = CreatInputExampleList(df_class)
    return samples

In [ ]:
train_class_samples = CreatClassSamples(train_class)
test_class_samples = CreatClassSamples(test_class)
dev_class_samples = CreatClassSamples(dev_class)

In [ ]:
train_dataloader_cl = DataLoader(train_class_samples, shuffle=True, batch_size=train_batch_size)
num_lables = test_class.select('annotator_labels').distinct().count()
train_loss_cl = losses.SoftmaxLoss(model=model, sentence_embedding_dimension=model.get_sentence_embedding_dimension(), num_labels=num_lables)

In [ ]:
evaluator_cl = EmbeddingSimilarityEvaluator.from_input_examples(dev_class_samples, batch_size=train_batch_size, name='snli-dev')
warmup_steps_cl = math.ceil(len(train_dataloader) * num_epochs * 0.1)

In [ ]:
model_class_location = './training_snli'

if os.path.exists(model_class_location):
    model = SentenceTransformer(model_class_location)
else:
    model.fit(train_objectives=[(train_dataloader_cl, train_loss_cl)],
             evaluator=evaluator_cl,
             epochs=num_epochs,
             evaluation_steps=1000,
             warmup_steps=warmup_steps_cl,
             output_path=model_class_location)

In [ ]:
evaluation_class_location = "./classification"

if not os.path.exists(evaluation_class_location):
    os.makedirs(evaluation_class_location)

test_eval_cl = EmbeddingSimilarityEvaluator.from_input_examples(test_class_samples, batch_size=train_batch_size, name='snli-test')
c_s_cl = test_eval_cl(model, output_path=evaluation_class_location)
print('Cosine similarity with the sentence_transformers library = ', c_s_cl)

In [ ]:
c_s_sts = test_eval(model, output_path=evaluation_class_location)
print('Cosine similarity with the sentence_transformers library = ', c_s_sts)

In [ ]:
embed_1_snli = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2_snli = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

embed_1 = model.encode(df_test['sentence_1'], convert_to_numpy=True, batch_size=train_batch_size)
embed_2 = model.encode(df_test['sentence_2'], convert_to_numpy=True, batch_size=train_batch_size)

In [ ]:
cos_sim_cl = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1_snli, embed_2_snli)
print('SNLI-test: cosine similarity = ', cos_sim_cl)

cos_sim_sts = 1 - sklearn.metrics.pairwise.paired_cosine_distances(embed_1, embed_2)
print('STS benchmark: cosine similarity = ', cos_sim_sts)

In [ ]:
spr_corr_cl = scipy.stats.spearmanr(cos_sim_cl, df_test['score'])
print('SNLI-test: Spearmean correlation coefficient = ', spr_corr_cl[0])

spr_corr_sts = scipy.stats.spearmanr(cos_sim_sts, df_test['score'])
print('STS benchmark: Spearmean correlation coefficient = ', spr_corr_sts[0])